In [1]:
# importing required libraries
import numpy as np
import pandas as pd
import pickle # saving and loading trained model
from os import path

# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# importing library for plotting
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from keras.layers import Dense, LSTM, MaxPool1D, Flatten, Dropout # importing dense layer
from keras.models import Sequential #importing Sequential layer
from keras.layers import Input
from keras.models import Model
# representation of model layers
from tensorflow.keras.utils import plot_model

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
train_data

,Duration,Protocol_type,Service,Flag,Src_bytes,Dst_bytes,Land,Wrong_fragment,Urgent,Hot,...,Dst_host_same_srv_rate,Dst_host_diff_srv_rate,Dst_host_same_src_port_rate,Dst_host_srv_diff_host_rate,Dst_host_serror_rate,Dst_host_srv_serror_rate,Dst_host_rerror_rate,Dst_host_srv_rerror_rate,attack_type,difficulty_level
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25186,0,tcp,exec,RSTO,0,0,0,0,0,0,...,0.03,0.06,0.00,0.00,0.00,0.00,1.0,1.00,neptune,19
25187,0,tcp,ftp_data,SF,334,0,0,0,0,0,...,1.00,0.00,1.00,0.18,0.00,0.00,0.0,0.00,warezclient,12
25188,0,tcp,private,REJ,0,0,0,0,0,0,...,0.05,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune,21
25189,0,tcp,nnsp,S0,0,0,0,0,0,0,...,0.08,0.06,0.00,0.00,1.00,1.00,0.0,0.00,neptune,20


In [4]:
test_data

,Duration,Protocol_type,Service,Flag,Src_bytes,Dst_bytes,Land,Wrong_fragment,Urgent,Hot,...,Dst_host_same_srv_rate,Dst_host_diff_srv_rate,Dst_host_same_src_port_rate,Dst_host_srv_diff_host_rate,Dst_host_serror_rate,Dst_host_srv_serror_rate,Dst_host_rerror_rate,Dst_host_srv_rerror_rate,attack_type,difficulty_level
0,13,tcp,telnet,SF,118,2425,0,0,0,0,...,0.38,0.12,0.04,0.00,0.00,0.00,0.12,0.30,guess_passwd,2
1,0,udp,private,SF,44,0,0,0,0,0,...,1.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,snmpguess,12
2,0,tcp,telnet,S3,0,44,0,0,0,0,...,0.31,0.61,0.00,0.00,0.21,0.68,0.60,0.00,processtable,18
3,0,udp,private,SF,53,55,0,0,0,0,...,1.00,0.00,0.87,0.00,0.00,0.00,0.00,0.00,normal,17
4,0,tcp,private,SH,0,0,0,0,0,0,...,0.06,1.00,1.00,0.00,1.00,1.00,0.00,0.00,nmap,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11845,0,udp,domain_u,SF,43,43,0,0,0,0,...,1.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,normal,18
11846,0,tcp,http,SF,336,285,0,0,0,0,...,0.92,0.02,0.00,0.00,0.00,0.00,0.05,0.00,normal,18
11847,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.37,0.03,0.01,0.02,0.05,0.08,0.85,0.58,mscan,13
11848,0,tcp,sunrpc,REJ,0,0,0,0,0,0,...,0.19,0.03,0.01,0.04,0.00,0.00,0.88,1.00,mscan,15


In [5]:
df= pd.concat([train_data, test_data], ignore_index=True)
df

,Duration,Protocol_type,Service,Flag,Src_bytes,Dst_bytes,Land,Wrong_fragment,Urgent,Hot,...,Dst_host_same_srv_rate,Dst_host_diff_srv_rate,Dst_host_same_src_port_rate,Dst_host_srv_diff_host_rate,Dst_host_serror_rate,Dst_host_srv_serror_rate,Dst_host_rerror_rate,Dst_host_srv_rerror_rate,attack_type,difficulty_level
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37036,0,udp,domain_u,SF,43,43,0,0,0,0,...,1.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,normal,18
37037,0,tcp,http,SF,336,285,0,0,0,0,...,0.92,0.02,0.00,0.00,0.00,0.00,0.05,0.00,normal,18
37038,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.37,0.03,0.01,0.02,0.05,0.08,0.85,0.58,mscan,13
37039,0,tcp,sunrpc,REJ,0,0,0,0,0,0,...,0.19,0.03,0.01,0.04,0.00,0.00,0.88,1.00,mscan,15


In [6]:
df = df.rename(columns={'attack_type': 'Label', 'difficulty_level': 'Difficulty'})
df

,Duration,Protocol_type,Service,Flag,Src_bytes,Dst_bytes,Land,Wrong_fragment,Urgent,Hot,...,Dst_host_same_srv_rate,Dst_host_diff_srv_rate,Dst_host_same_src_port_rate,Dst_host_srv_diff_host_rate,Dst_host_serror_rate,Dst_host_srv_serror_rate,Dst_host_rerror_rate,Dst_host_srv_rerror_rate,Label,Difficulty
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37036,0,udp,domain_u,SF,43,43,0,0,0,0,...,1.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,normal,18
37037,0,tcp,http,SF,336,285,0,0,0,0,...,0.92,0.02,0.00,0.00,0.00,0.00,0.05,0.00,normal,18
37038,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.37,0.03,0.01,0.02,0.05,0.08,0.85,0.58,mscan,13
37039,0,tcp,sunrpc,REJ,0,0,0,0,0,0,...,0.19,0.03,0.01,0.04,0.00,0.00,0.88,1.00,mscan,15


In [7]:
df.drop(['Difficulty'],axis=1,inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37041 entries, 0 to 37040
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Duration                     37041 non-null  int64  
 1   Protocol_type                37041 non-null  object 
 2   Service                      37041 non-null  object 
 3   Flag                         37041 non-null  object 
 4   Src_bytes                    37041 non-null  int64  
 5   Dst_bytes                    37041 non-null  int64  
 6   Land                         37041 non-null  int64  
 7   Wrong_fragment               37041 non-null  int64  
 8   Urgent                       37041 non-null  int64  
 9   Hot                          37041 non-null  int64  
 10  Num_failed_logins            37041 non-null  int64  
 11  Logged_in                    37041 non-null  int64  
 12  Num_compromised              37041 non-null  int64  
 13  Root_shell      

In [9]:
df.columns

Index(['Duration', 'Protocol_type', 'Service', 'Flag', 'Src_bytes',
       'Dst_bytes', 'Land', 'Wrong_fragment', 'Urgent', 'Hot',
       'Num_failed_logins', 'Logged_in', 'Num_compromised', 'Root_shell',
       'Su_attempted', 'Num_root', 'Num_file_creations', 'Num_shells',
       'Num_access_files', 'Num_outbound_cmds', 'Is_hot_login',
       'Is_guest_login', 'Count', 'Srv_count', 'Serror_rate',
       'Srv_serror_rate', 'Rerror_rate', 'Srv_rerror_rate', 'Same_srv_rate',
       'Diff_srv_rate', 'Srv_diff_host_rate', 'Dst_host_count',
       'Dst_host_srv_count', 'Dst_host_same_srv_rate',
       'Dst_host_diff_srv_rate', 'Dst_host_same_src_port_rate',
       'Dst_host_srv_diff_host_rate', 'Dst_host_serror_rate',
       'Dst_host_srv_serror_rate', 'Dst_host_rerror_rate',
       'Dst_host_srv_rerror_rate', 'Label'],
      dtype='object')

In [10]:
df.isnull().sum()

Duration                       0
Protocol_type                  0
Service                        0
Flag                           0
Src_bytes                      0
Dst_bytes                      0
Land                           0
Wrong_fragment                 0
Urgent                         0
Hot                            0
Num_failed_logins              0
Logged_in                      0
Num_compromised                0
Root_shell                     0
Su_attempted                   0
Num_root                       0
Num_file_creations             0
Num_shells                     0
Num_access_files               0
Num_outbound_cmds              0
Is_hot_login                   0
Is_guest_login                 0
Count                          0
Srv_count                      0
Serror_rate                    0
Srv_serror_rate                0
Rerror_rate                    0
Srv_rerror_rate                0
Same_srv_rate                  0
Diff_srv_rate                  0
Srv_diff_h

In [11]:
df['Label'].value_counts()

normal             15600
neptune             9861
satan               1418
guess_passwd        1241
smurf               1156
mscan                996
warezmaster          951
ipsweep              851
portsweep            743
apache2              737
processtable         685
back                 555
nmap                 374
snmpguess            331
saint                309
mailbomb             293
teardrop             200
warezclient          181
snmpgetattack        178
httptunnel           133
pod                   79
buffer_overflow       26
multihop              20
named                 17
rootkit               17
ps                    15
sendmail              14
xterm                 13
xlock                  9
land                   8
imap                   6
phf                    4
xsnoop                 4
ftp_write              4
loadmodule             3
worm                   2
udpstorm               2
perl                   2
sqlattack              2
spy                    1


In [12]:
# Redistribute across common attack class
def change_label(df):
  df.Label.replace(['apache2','back','land','neptune','mailbomb','pod','processtable','smurf','teardrop','udpstorm','worm'],'Dos',inplace=True)
  df.Label.replace(['ftp_write','guess_passwd','httptunnel','imap','multihop','named','phf','sendmail','snmpgetattack','snmpguess','spy','warezclient','warezmaster','xlock','xsnoop'],'R2L',inplace=True)      
  df.Label.replace(['ipsweep','mscan','nmap','portsweep','saint','satan'],'Probe',inplace=True)
  df.Label.replace(['buffer_overflow','loadmodule','perl','ps','rootkit','sqlattack','xterm'],'U2R',inplace=True)

In [13]:
change_label(df)

In [14]:
df

,Duration,Protocol_type,Service,Flag,Src_bytes,Dst_bytes,Land,Wrong_fragment,Urgent,Hot,...,Dst_host_srv_count,Dst_host_same_srv_rate,Dst_host_diff_srv_rate,Dst_host_same_src_port_rate,Dst_host_srv_diff_host_rate,Dst_host_serror_rate,Dst_host_srv_serror_rate,Dst_host_rerror_rate,Dst_host_srv_rerror_rate,Label
0,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
1,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,Dos
2,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
3,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
4,0,tcp,private,REJ,0,0,0,0,0,0,...,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,Dos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37036,0,udp,domain_u,SF,43,43,0,0,0,0,...,255,1.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,normal
37037,0,tcp,http,SF,336,285,0,0,0,0,...,234,0.92,0.02,0.00,0.00,0.00,0.00,0.05,0.00,normal
37038,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,96,0.37,0.03,0.01,0.02,0.05,0.08,0.85,0.58,Probe
37039,0,tcp,sunrpc,REJ,0,0,0,0,0,0,...,52,0.19,0.03,0.01,0.04,0.00,0.00,0.88,1.00,Probe


In [15]:
df['Label'].value_counts()

normal    15600
Dos       13578
Probe      4691
R2L        3094
U2R          78
Name: Label, dtype: int64

In [16]:
Label = pd.DataFrame(df.Label)

In [17]:
std_scaler = MinMaxScaler()
def standardization(df,col):
    for i in col:
        arr = df[i]
        arr = np.array(arr)
        df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    return df

numeric_col = df.select_dtypes(include='number').columns
df = standardization(df,numeric_col)

In [18]:
# label encoding (0,1,2,3,4) multi-class labels (Dos,normal,Probe,R2L,U2R)
le2 = preprocessing.LabelEncoder()
enc_label = Label.apply(le2.fit_transform)
df['intrusion'] = enc_label
print(df.shape)
df

(37041, 43)


,Duration,Protocol_type,Service,Flag,Src_bytes,Dst_bytes,Land,Wrong_fragment,Urgent,Hot,...,Dst_host_same_srv_rate,Dst_host_diff_srv_rate,Dst_host_same_src_port_rate,Dst_host_srv_diff_host_rate,Dst_host_serror_rate,Dst_host_srv_serror_rate,Dst_host_rerror_rate,Dst_host_srv_rerror_rate,Label,intrusion
0,0.000000,udp,other,SF,3.824902e-07,0.000000,0.0,0.0,0.0,0.0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,4
1,0.000000,tcp,private,S0,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,Dos,0
2,0.000000,tcp,http,SF,6.077927e-07,0.001583,0.0,0.0,0.0,0.0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,4
3,0.000000,tcp,http,SF,5.213394e-07,0.000082,0.0,0.0,0.0,0.0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,4
4,0.000000,tcp,private,REJ,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,Dos,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37036,0.000000,udp,domain_u,SF,1.126512e-07,0.000008,0.0,0.0,0.0,0.0,...,1.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,normal,4
37037,0.000000,tcp,http,SF,8.802515e-07,0.000055,0.0,0.0,0.0,0.0,...,0.92,0.02,0.00,0.00,0.00,0.00,0.05,0.00,normal,4
37038,0.000017,tcp,telnet,RSTO,0.000000e+00,0.000003,0.0,0.0,0.0,0.0,...,0.37,0.03,0.01,0.02,0.05,0.08,0.85,0.58,Probe,1
37039,0.000000,tcp,sunrpc,REJ,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,...,0.19,0.03,0.01,0.04,0.00,0.00,0.88,1.00,Probe,1


In [19]:
df.drop(labels= ['Label'], axis=1, inplace=True)
print(df.shape)

(37041, 42)


In [20]:
# one-hot-encoding categorical columns
df = pd.get_dummies(df,columns=['Protocol_type','Service','Flag'],prefix="",prefix_sep="")  
print(df.shape)

(37041, 120)


In [21]:
y_data= df['intrusion']
X_data= df.drop(labels=['intrusion'], axis=1)

In [22]:
X_data = X_data.astype(np.float32)
Y_data = y_data.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X_data,y_data, test_size=0.20, random_state=42)
print(X_train.shape,'\n',X_test.shape)

(29632, 119) 
 (7409, 119)


In [23]:
from sklearn.feature_selection import SelectFromModel

sel = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1))
sel.fit(X_train, y_train)
sel.get_support()

X_train.columns

features = X_train.columns[sel.get_support()]
print(features)

print(len(features))

np.mean(sel.estimator_.feature_importances_)
print(sel.estimator_.feature_importances_)

<IPython.core.display.Javascript object>

Index(['Duration', 'Src_bytes', 'Dst_bytes', 'Hot', 'Logged_in', 'Count',
       'Srv_count', 'Serror_rate', 'Srv_serror_rate', 'Rerror_rate',
       'Srv_rerror_rate', 'Same_srv_rate', 'Diff_srv_rate',
       'Srv_diff_host_rate', 'Dst_host_count', 'Dst_host_srv_count',
       'Dst_host_same_srv_rate', 'Dst_host_diff_srv_rate',
       'Dst_host_same_src_port_rate', 'Dst_host_srv_diff_host_rate',
       'Dst_host_serror_rate', 'Dst_host_srv_serror_rate',
       'Dst_host_rerror_rate', 'Dst_host_srv_rerror_rate', 'icmp', 'tcp',
       'udp', 'eco_i', 'ecr_i', 'http', 'pop_3', 'S0', 'SF'],
      dtype='object')
33
[2.09564406e-02 3.88128096e-02 7.56408263e-02 4.43256425e-05
 3.90636857e-03 1.45080039e-04 1.05647110e-02 6.16775228e-03
 2.12423697e-02 5.40708891e-03 4.12056775e-04 6.86001758e-05
 7.93499604e-04 5.07356917e-04 2.63110707e-04 2.65870999e-04
 0.00000000e+00 1.23102309e-04 5.53616524e-03 5.09232635e-02
 3.09880814e-02 2.40804058e-02 2.83556444e-02 1.64953923e-02
 1.25517438e-0

In [24]:
X_train_rfc = sel.transform(X_train)
X_test_rfc = sel.transform(X_test)

X_train_rfc=np.array(X_train_rfc)
X_test_rfc=np.array(X_test_rfc)

from sklearn.preprocessing import LabelBinarizer
y_train = LabelBinarizer().fit_transform(y_train)
y_test = LabelBinarizer().fit_transform(y_test)

In [25]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train_rfc, ( X_train_rfc.shape[0], 1 , X_train_rfc.shape[1] ))
X_test = np.reshape(X_test_rfc, ( X_test_rfc.shape[0], 1,  X_test_rfc.shape[1] ))

In [26]:
model = Sequential() # initializing model
model.add(LSTM(64,return_sequences=True,input_shape = (1, X_train_rfc.shape[1])))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Flatten())
model.add(Dense(units=50))
# output layer with softmax activation
model.add(Dense(units=5,activation='softmax'))

In [27]:
# defining loss function, optimizer, metrics and then compiling model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
# summary of model layers
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 64)             25088     
                                                                 
 dropout (Dropout)           (None, 1, 64)             0         
                                                                 
 lstm_1 (LSTM)               (None, 1, 64)             33024     
                                                                 
 dropout_1 (Dropout)         (None, 1, 64)             0         
                                                                 
 lstm_2 (LSTM)               (None, 1, 64)             33024     
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 50)                3

In [29]:
history = model.fit(X_train, y_train, epochs=100, batch_size=5000,validation_split=0.2)

Epoch 1/100


5/5 [==============================] - 8s 456ms/step - loss: 1.5986 - accuracy: 0.3935 - val_loss: 1.5792 - val_accuracy: 0.4238
Epoch 2/100
5/5 [==============================] - 0s 83ms/step - loss: 1.5663 - accuracy: 0.4415 - val_loss: 1.5416 - val_accuracy: 0.4238
Epoch 3/100
5/5 [==============================] - 0s 93ms/step - loss: 1.5249 - accuracy: 0.4358 - val_loss: 1.4911 - val_accuracy: 0.4238
Epoch 4/100
5/5 [==============================] - 1s 112ms/step - loss: 1.4683 - accuracy: 0.4310 - val_loss: 1.4203 - val_accuracy: 0.4238
Epoch 5/100
5/5 [==============================] - 1s 148ms/step - loss: 1.3897 - accuracy: 0.4256 - val_loss: 1.3236 - val_accuracy: 0.4238
Epoch 6/100
5/5 [==============================] - 1s 176ms/step - loss: 1.2868 - accuracy: 0.4231 - val_loss: 1.2127 - val_accuracy: 0.4238
Epoch 7/100
5/5 [==============================] - 1s 163ms/step - loss: 1.1904 - accuracy: 0.4273 - val_loss: 1.1343 - val_accuracy: 0.4240
Epoch 8/100
5

In [30]:
test_results = model.evaluate(X_test, y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')

232/232 [==============================] - 2s 9ms/step - loss: 0.1886 - accuracy: 0.9300
Test results - Loss: 0.18863745033740997 - Accuracy: 92.99500584602356%
